In [5]:
import sys
from io import BytesIO
sys.path.append('../..')
from functions.select_cols_functions import *
from functions.aws_functions import *
import util.credentials as u

In [6]:
bucket_name = 'adoptai-raw-prod'
new_bucket_name = 'adoptai-trusted-prod'

In [7]:
session = boto3.Session(
    aws_access_key_id= u.aws_access_key_id,
    aws_secret_access_key= u.aws_secret_access_key,
    aws_session_token= u.aws_session_token
)

s3_client = session.client('s3')

In [8]:
try:
    s3_csv = s3_csv_to_df(bucket_name)
    
    object_key = s3_csv[0]
    df = s3_csv[1]

    df_treated = apply_all_rules(df)

    csv_buffer = BytesIO()
    df_treated.to_csv(csv_buffer, index=False)

    s3_client.put_object(Bucket=new_bucket_name, Key=object_key, Body=csv_buffer.getvalue())
    print(f'Novo arquivo salvo em {new_bucket_name}/{object_key}')
except Exception as e:
    print(f'Erro ao gravar o arquivo no s3 destino: {e}')


Último arquivo adicionado: train_2_classes.csv
Novo arquivo salvo em adoptai-trusted-prod/train_2_classes.csv
